# Brazilian senate minutes scraping

*Author: Danilo Lessa Bernardineli (danilo.bernardineli@usp.br)*

In [1]:
import requests as req
import re
import os
from pathlib import Path

from tqdm.notebook import tqdm
from unidecode import unidecode
from bs4 import BeautifulSoup

In [2]:
BASE_URI = 'http://www.senado.leg.br/publicacoes/anais'
SOURCE_URIS = ['http://www.senado.leg.br/publicacoes/anais/asp/IP_AnaisImperio_digitalizados.asp',
               'http://www.senado.leg.br/publicacoes/anais/asp/RP_AnaisRepublica.asp']
OUTPUT_PATH = "../working_dir/scrap/senate/"

In [3]:
elements = []
for uri in SOURCE_URIS:
    r = req.get(uri)
    r.encoding = r.apparent_encoding
    soup = BeautifulSoup(r.text)
    soup_items = soup.select('#conteudoPrincipal > div:nth-child(3) > div  li')
    elements += soup_items

In [4]:
def clean_link_text(string: str) -> str:
    return unidecode(string.lower().replace(" ", "_"))

link_dict = {}
for bullet in elements:
    bullet_text = bullet.find(text=True, recursive=False).strip()
    year = int(re.sub('\D', '', bullet_text))
    links = {clean_link_text(link.text): link['href'].replace("..", BASE_URI)
             for link in bullet.select('a')}
    link_dict[year] = links

In [5]:
download_map = {"{}_{}.pdf".format(year, label): uri
                for year, files in link_dict.items()
                for label, uri in files.items()}

In [ ]:
for filename, uri in tqdm(download_map.items()):
    filepath = Path(OUTPUT_PATH, filename)
    content = req.get(uri).content
    with open(filepath, 'wb') as fid:
        fid.write(content)